In [1]:
from sklearn.utils import shuffle
import cv2
import os
import numpy as np
from keras.layers import Dropout
from keras import applications
from keras.layers import Dense, GlobalAveragePooling2D, merge, Input
from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
import glob
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import imageio as im
from keras import models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.applications import vgg16
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Activation,GlobalAveragePooling2D,concatenate
from keras.applications.mobilenet import MobileNet
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup
from os.path import abspath
import urllib
import pandas as pd
import pickle 
import numpy as np
import xgboost as xgb
import os
from keras.models import load_model
import shutil
import cv2
image_size=128
import os
import tensorflow as tf
import cv2
import glob
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
import numpy
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Input,Dense, Conv2D, Dropout, Flatten, MaxPooling2D,Embedding,LSTM
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import os
os.listdir('/kaggle/input/text-image/')

['10k_tested.xlsx',
 'Training_Data',
 'LSTM_text.h5',
 'Human_Organisation_Bios.xlsx',
 'shapes_cnn_mobile.h5']

In [3]:
train_path=r'/kaggle/input/text-image/Training_Data/Training_Data/'
image_size=128
classes=os.listdir(train_path)
def load_train(train_path, image_size, classes):
    images = []
    labels = []
    img_names = []

    print('Going to read training images')
    for fields in classes:   
        index = classes.index(fields)
        print('Now going to read {} files (Index: {})'.format(fields, index))
        path = os.path.join(train_path, fields, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            #image = np.multiply(image, 1.0 / 255.0)
            images.append(image)
            label = np.zeros(len(classes))
            label[index] = 1.0
            labels.append(label)
            #labels.append(fields)#for regression
            flbase = os.path.basename(fl)
            img_names.append(flbase)
    images = np.array(images)
    labels = np.array(labels)
# =============================================================================
#     labels=np.reshape(labels,(-1,1))
# =============================================================================
    img_names = np.array(img_names)

    return images, labels, img_names


In [4]:
images, labels, img_names = load_train(train_path, image_size, classes)
images, labels, img_names = shuffle(images, labels, img_names) 
x_train,x_test,y_train,y_test=train_test_split(images,labels, test_size=0.33)
x_train = x_train.reshape(x_train.shape[0], 128, 128, 3)
x_test = x_test.reshape(x_test.shape[0], 128, 128, 3) 

Going to read training images
Now going to read Humans files (Index: 0)
Now going to read Organisations files (Index: 1)


In [5]:
train=pd.read_excel(r'/kaggle/input/text-image/Human_Organisation_Bios.xlsx')
train=train.replace('Human','Humans')
train=train.replace('Organisation','Organisations')
enc = LabelEncoder().fit(train['Classification'])
encoded = enc.transform(train['Classification'])
encoded = keras.utils.to_categorical(encoded)
test1=encoded
train=train.drop(['Classification'],axis=1)
for i in train:
    train[i]=train[i].astype(str)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['Bios'])

X_train = tokenizer.texts_to_sequences(train['Bios'])
# X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = 1024

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [6]:
e_model=load_model('/kaggle/input/text-image/shapes_cnn_mobile.h5')
intermediate_layer_model = Model(inputs=e_model.input,outputs=e_model.get_layer('dense_1').output)

In [7]:
e_model2=load_model('/kaggle/input/text-image/LSTM_text.h5')
intermediate_layer_model2 = Model(inputs=e_model2.input,outputs=e_model2.get_layer('FC1').output)

In [8]:
max_words = 1024
epochs = 100
batch_size = 32

X_train_image = images
X_train_text = X_train
y_train = test1

#num_classes = np.max(y_train) + 1
#print(num_classes)
# Text input branch - just a simple MLP
text_inputs = Input(shape=(max_words,))
# # branch_1 = Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.01))(text_inputs)
# # branch_1=BatchNormalization()(branch_1)
# branch_1 = Embedding(max_words,50,input_length=maxlen)(text_inputs)
# branch_1 = LSTM(64)(branch_1)
# branch_1 = Dense(256,name='FC1')(branch_1)

# # Image input branch - a pre-trained Inception module followed by an added fully connected layer
# #base_model = applications.InceptionV3(weights='imagenet', include_top=False)
# base_model=applications.MobileNet(weights='imagenet',include_top=False)
# # Freeze Inception's weights - we don't want to train these
# for layer in base_model.layers:
#     layer.trainable = False


# # add a fully connected layer after Inception - we do want to train these
# branch_2 = base_model.output
# branch_2 = GlobalAveragePooling2D()(branch_2)
# branch_2 = Dense(1024, activation='relu')(branch_2)
# branch_2 = BatchNormalization()(branch_2)
# merge the text input branch and the image input branch and add another fully connected layer
joint = concatenate([intermediate_layer_model2.output, intermediate_layer_model.output])
joint = Dense(512, activation='relu')(joint)
joint=BatchNormalization()(joint)
joint = Dropout(0.7)(joint)
predictions = Dense(2, activation='softmax')(joint)

full_model = Model(inputs=[intermediate_layer_model2.input,intermediate_layer_model.input], outputs=[predictions])

opt=Adam(learning_rate=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)


full_model.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history = full_model.fit([X_train_text, X_train_image], y_train,
                         epochs=epochs, batch_size=batch_size,
                         verbose=1, validation_split=0.3, shuffle=True)
full_model.save('text_image2_lr.h5')

ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.

In [9]:
max_words = 1024
epochs = 100
batch_size = 32

X_train_image = images
X_train_text = X_train
y_train = test1

#num_classes = np.max(y_train) + 1
#print(num_classes)
# Text input branch - just a simple MLP
text_inputs = Input(shape=(max_words,))
# # branch_1 = Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.01))(text_inputs)
# # branch_1=BatchNormalization()(branch_1)
# branch_1 = Embedding(max_words,50,input_length=maxlen)(text_inputs)
# branch_1 = LSTM(64)(branch_1)
# branch_1 = Dense(256,name='FC1')(branch_1)

# # Image input branch - a pre-trained Inception module followed by an added fully connected layer
# #base_model = applications.InceptionV3(weights='imagenet', include_top=False)
# base_model=applications.MobileNet(weights='imagenet',include_top=False)
# # Freeze Inception's weights - we don't want to train these
# for layer in base_model.layers:
#     layer.trainable = False


# # add a fully connected layer after Inception - we do want to train these
# branch_2 = base_model.output
# branch_2 = GlobalAveragePooling2D()(branch_2)
# branch_2 = Dense(1024, activation='relu')(branch_2)
# branch_2 = BatchNormalization()(branch_2)
# merge the text input branch and the image input branch and add another fully connected layer
joint = concatenate([intermediate_layer_model2.output, intermediate_layer_model.output])
joint = Dense(512, activation='relu')(joint)
joint=BatchNormalization()(joint)
joint = Dropout(0.7)(joint)
predictions = Dense(2, activation='softmax')(joint)

full_model = Model(inputs=[intermediate_layer_model2.input,intermediate_layer_model.input], outputs=[predictions])

opt=Adam(learning_rate=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)


full_model.compile(loss='categorical_crossentropy',
                   optimizer=opt,
                   metrics=['accuracy'])

history = full_model.fit([X_train_text, X_train_image], y_train,
                         epochs=epochs, batch_size=batch_size,
                         verbose=1, validation_split=0.3, shuffle=True)
full_model.save('text_image2_lr.h5')

Train on 5908 samples, validate on 2532 samples
Epoch 1/100
5908/5908 [==============================] - 232s 39ms/step - loss: 1.4486 - accuracy: 0.5184 - val_loss: 0.8802 - val_accuracy: 0.2085
Epoch 2/100
5908/5908 [==============================] - 227s 38ms/step - loss: 1.4162 - accuracy: 0.5103 - val_loss: 0.8612 - val_accuracy: 0.2859
Epoch 3/100
5908/5908 [==============================] - 228s 39ms/step - loss: 1.3486 - accuracy: 0.5171 - val_loss: 0.8246 - val_accuracy: 0.3187
Epoch 4/100
5908/5908 [==============================] - 228s 39ms/step - loss: 1.3094 - accuracy: 0.5245 - val_loss: 0.8023 - val_accuracy: 0.3479
Epoch 5/100
5908/5908 [==============================] - 227s 38ms/step - loss: 1.2894 - accuracy: 0.5044 - val_loss: 0.7864 - val_accuracy: 0.3543
Epoch 6/100
5908/5908 [==============================] - 227s 38ms/step - loss: 1.2796 - accuracy: 0.5081 - val_loss: 0.7761 - val_accuracy: 0.3645
Epoch 7/100
5908/5908 [==============================] - 226s 38

In [10]:
# x=pd.DataFrame(columns=['Bios'])
# x['Bios']=['Creative available for commission work. Specializing in concept development, illustration, murals, graphics and 3-D set design & construction.']
# max_len = 100
# cnn_texts_seq = tokenizer.texts_to_sequences(x['Bios'])
# print(cnn_texts_seq[0])
# cnn_texts_mat = pad_sequences(cnn_texts_seq,maxlen=max_len)

In [11]:
# y=[]
# for i in df['ScreenName']:
#     link='https://twitter.com/'+i
#     try:
#         html_page = urllib.request.urlopen(link,timeout=10)
#     except (urllib.error.HTTPError,urllib.error.URLError):
#         print('I cannot handle this situation!')
#     soup = BeautifulSoup(html_page)
#     if os.path.exists('twitter'):
#         pass
#     else:
#         os.mkdir('twitter')
#     urllib.request.urlretrieve(soup.find(class_="ProfileAvatar-image").get('src'),  r'/kaggle/input/twitter'+"//"+link.split('/')[3]+'.jpg')
#     text=soup.find(class_="ProfileHeaderCard-bio u-dir").get_text()
#     x=pd.DataFrame(columns=['Bios'])
#     x['Bios']=["'"+text+"'"]
#     max_len = 100
#     cnn_texts_seq = tokenizer.texts_to_sequences(x['Bios'])
#     cnn_texts_mat = pad_sequences(cnn_texts_seq,maxlen=max_len)
#     images=[]
#     image = cv2.imread(r'/kaggle/input/twitter'+"//"+link.split('/')[3]+'.jpg')
#     image1 = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
#     images.append(image1)
#     images = np.array(images, dtype=np.uint8)
#     images = images.astype('float32')
#     #images = np.multiply(images, 1.0/255.0)
#     print(model.predict([cnn_texts_mat,images]))
# #     for i in model.predict([cnn_texts_mat,images]):
# #             if i<0.5:
# #                 print('It is a human profile')
# #                 y.append('Human')
# #             else:
# #                 print('It is an organisation profile')
# #                 y.append('Organisation')
#     shutil.rmtree('twitter')

In [12]:
# link=input('')
# try:
#     html_page = urllib.request.urlopen(link,timeout=10)
# except (urllib.error.HTTPError,urllib.error.URLError):
#     print('I cannot handle this situation!')
# soup = BeautifulSoup(html_page)
# if os.path.exists('twitter'):
#     pass
# else:
#     os.mkdir('twitter')
# urllib.request.urlretrieve(soup.find(class_="ProfileAvatar-image").get('src'),  r'/kaggle/working/twitter'+"/"+link.split('/')[3]+'.jpg')
# text=soup.find(class_="ProfileHeaderCard-bio u-dir").get_text()
# x=pd.DataFrame(columns=['Bios'])
# x['Bios']=["'"+text+"'"]
# max_len = 1024
# cnn_texts_seq = tokenizer.texts_to_sequences(x['Bios'])
# cnn_texts_mat = pad_sequences(cnn_texts_seq,maxlen=max_len)
# image2=[]
# image = cv2.imread(r'/kaggle/working/twitter'+"/"+link.split('/')[3]+'.jpg')
# image1 = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
# image2.append(image1)
# image2 = np.array(image2, dtype=np.uint8)
# image2 = image2.astype('float32')
# #images = np.multiply(images, 1.0/255.0)
# print(np.argmax(full_model.predict([cnn_texts_mat,image2])))
